# Cnvolutional neural network Signal: Tutorial
- 실습조교: 배진수(wlstn215@korea.ac.kr), 김정인(jungin_kim23@korea.ac.kr), 정진용(jy_jeong@korea.ac.kr)

## Colab gpu 연결

#### 런타임 -> 런타임유형 변경 -> 하드웨어 가속기 -> GPU

In [ ]:
import torch, os
torch.cuda.is_available()

## 0.모듈 불러오기

In [ ]:
''' 기본 모듈 및 시각화 모듈'''
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle

'''CNN을 위한 딥러닝 모듈'''
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

''' 결과 평가용 모듈 '''
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

''' github+colab 교육생분들 '''
# !git clone https://github.com/bogus215/LG-EDUCATION2.git

### <GPU 확인, 사용할 device 설정>

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### <Seed/Random state 고정, 재현성을 위해>

In [ ]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

## 1. 데이터 불러오기: Human Activity Recognition Data
https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones#
- Human Activity Recognition (HAR) Data는 30명의 실험자들에 대해서 각자 스마트폰을 허리에 착용하고 6가지 활동 (Walking, Walking Upstairs, Walking Downstairs, Sitting, Standing, Laying)을 수행할 때 측정된 센서 데이터로 구성된 데이터셋
- 총 9종류의 센서로 이루어진 데이터이며, 각 센서는 50Hz 단위로 측정이 이루어짐.
- 2.56초에 해당하는 window_size를 적용하여 다변량 센서 데이터를 입력 변수(X)로, 활동을 출력 변수(Y)로 분류하는 것이 목표

<img src="https://github.com/bogus215/LG-EDUCATION2/blob/main/image/image4.png?raw=1" width="600">

<img src="https://github.com/bogus215/LG-EDUCATION2/blob/main/image/image5.png?raw=1" width="600">

### - 데이터 전처리

In [ ]:
# txt 파일로 존재하는 데이터 line 별로 읽어오기
# (num_data,sequence_length)의 data 생성

def create_data(path, filename):
    with open(os.path.join(path, filename)) as f:
        data = []
        for line in f:
            num = [float(l) for l in line.split()]
            data.append(num)
    data = np.array(data).reshape(-1,128)

    return data

In [ ]:
train_path = './data/UCI HAR Dataset/train/Inertial Signals'
test_path = './data/UCI HAR Dataset/test/Inertial Signals'
train_dir_lst = sorted(os.listdir(train_path))
test_dir_lst = sorted(os.listdir(test_path))

''' github+colab 교육생분들 '''
# train_path = './LG-EDUCATION2/data/UCI HAR Dataset/train/Inertial Signals'
# test_path = './LG-EDUCATION2/data/UCI HAR Dataset/test/Inertial Signals'

In [ ]:
# Conv1d 의 input 형태 : (batch_size,num_channels,sequence_length)
# 각 센서별로 나오는 데이터를 (num_data,num_channels,sequence_length)형태로 변경
train_data = []
for i in range(len(train_dir_lst)):
    data = create_data(train_path, train_dir_lst[i])
    train_data.append(data)
train_data = np.transpose(np.array(train_data),(1,0,2))

test_data = []
for i in range(len(test_dir_lst)):
    data = create_data(test_path, test_dir_lst[i])
    test_data.append(data)
test_data = np.transpose(np.array(test_data),(1,0,2))

In [ ]:
# 1~6 으로 class label이 형성되어 있으므로 0~5까지 맞춰주기 위해서 -1 
# 0(Walking) / 1(Walking Upstairs) / 2(Walking Downstairs) / 3(Sitting) / 4(Standing) / 5(Laying)
train_label = pd.read_csv('./data/UCI HAR Dataset/train/y_train.txt',header=None,sep=' ')
train_label = np.array(train_label[0])-1

test_label = pd.read_csv('./data/UCI HAR Dataset/test/y_test.txt',header=None,sep=' ')
test_label = np.array(test_label[0])-1

''' github+colab 교육생분들 '''
# train_label = pd.read_csv('./LG-EDUCATION2/data/UCI HAR Dataset/train/y_train.txt',header=None,sep=' ')
# test_label = pd.read_csv('./LG-EDUCATION2/data/UCI HAR Dataset/test/y_test.txt',header=None,sep=' ')

In [ ]:
# train 데이터를 활용하여 val 데이터 생성 ( train : val = 8 : 2)
len_train = int(len(train_data) * 0.8)

new_train_data = train_data[:len_train,:,:]
new_train_label = train_label[:len_train]

val_data = train_data[len_train:,:,:]
val_label = train_label[len_train:]

### Dataset and DataLoader
- Tensor : 데이터를 담고있는 다차원 행렬
- Dataset : 데이터를 불러오고 전처리 하는 과정
- Data Loader : Dataset을 입력받아 배치 사이즈에 맞추어 출력

In [ ]:
train_dataset = torch.utils.data.TensorDataset(torch.Tensor(new_train_data),torch.Tensor(new_train_label))
val_dataset = torch.utils.data.TensorDataset(torch.Tensor(val_data),torch.Tensor(val_label))
test_dataset = torch.utils.data.TensorDataset(torch.Tensor(test_data),torch.Tensor(test_label))

In [ ]:
# train/val/test를 위한 dataloader 생성
# input data shape : (batch_size,num_channels,sequence_length)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100)

## 2. Model: 1D CNN

### __Conv1d__ : (in_channels,out_channels,kernel_size)
- in_channels : input으로 들어오는 데이터의 채널 개수(다변량 시계열 데이터에서는 센서의 개수)
- out_channels : Convolution 연산 이후 만들어지는 채널의 개수
- kernel_size : Filter의 크기를 의미하며 하나의 Filter에서 고려할 시점의 개수

### Model
- torch.nn.Module 상속받기
- \_\_init\_\_: 모델에서 사용할 layer 정의
- forward: 데이터를 입력받아 모델 진행 순서 결정

<img src="https://github.com/bogus215/LG-EDUCATION2/blob/main/image/image8.png?raw=1" width="800">

In [ ]:
class CNN_1d(nn.Module):
    def __init__(self,num_classes=6):
        super().__init__()
        
        '''모델에 필요한 layer 정의'''
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(9,196,kernel_size=6),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size = 2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(196,64,kernel_size =6),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        
        self.classifier = nn.Linear(64*28,num_classes)
        
    def forward(self,x):
        
        ''' x를 입력받아 진행할 순서 설정 '''
        
        out = self.conv1(x) 
        out = self.conv2(out) 
        out = out.view(out.size(0),-1)
        out = self.classifier(out)
        
        return out

In [ ]:
model = CNN_1d()
model

## 3. 모델 학습

### 비용함수

In [ ]:
# 비용함수 정의 (분류문제 -> crossentropy 사용)
criterion = nn.CrossEntropyLoss()

### - Optimizer
- __params__ : update를 진행할 모델의 파라미터
- __lr__ : learning rate

In [ ]:
# optimizer로 Adam 활용
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# 모델을 'gpu' or 'cpu'로 이동
model.to(device)

num_epochs = 20

print("Start Training !")
print('-'*50)
print()

train_loss_total = []
val_loss_total = []
best_loss = np.inf

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)
    
    train_loss = 0
    val_loss = 0
    train_corrects = 0
    val_corrects = 0
    
    
    '''Train'''
    # 모델을 학습모드로 전환
    model.train()
    
    for inputs,labels in train_loader:
        # 데이터를 'gpu' or 'cpu' 로 이동
        inputs = inputs.to(device)
        labels = labels.to(device).long()
        
        
        # 모델의 모든 기울기 값을 0으로 설정
        optimizer.zero_grad()
        
        # 데이터를 모델에 입력하여 출력값 계산
        outputs = model(inputs)
        
        #outputs값 중에서 큰 class값을 가져옴
        _,preds = torch.max(outputs,1)
        
        # 비용함수를 활용하여 오차 계산
        loss = criterion(outputs,labels)
        
        # 계산된 오차를 기반으로 기울기 계산
        loss.backward()
        
        # 계산된 기울기를 바탕으로 모델의 파라미터 업데이트
        optimizer.step()
        
        train_loss += loss.item()
        train_corrects += torch.sum(preds == labels.data)
    
    '''Validation'''
    # 모델을 평가모드로 전환
    model.eval()
    # 모델을 평가할때는 기울기 계산 불필요
    with torch.no_grad():
        for inputs,labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).long()
            
            outputs = model(inputs)
            _,preds = torch.max(outputs,1)
            
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            val_corrects += torch.sum(preds == labels.data)
    
    # epoch의 loss 계산
    train_loss_epoch = train_loss/len(train_loader)
    val_loss_epoch = val_loss/len(val_loader)
    
    #epoch의 accuracy 계산
    train_acc_epoch = train_corrects / len(train_dataset)
    val_acc_epoch = val_corrects / len(val_dataset)

    print(f'{epoch+1} epoch | Train loss: {train_loss_epoch:.3f}, Valid loss: {val_loss_epoch:.3f}')
    print(f'{epoch+1} epoch | Train acc: {train_acc_epoch:.3f}, Valid acc: {val_acc_epoch:.3f}')
    print()

    # validation loss 기준으로 best epoch 계산 (Early stopping)
    if val_loss_epoch < best_loss:
        # best loss 업데이트
        best_loss = val_loss_epoch
        # 최적의 epoch 수와 모델 저장하기
        best_epoch = epoch
        torch.save(model.state_dict(), './best_model_HAR.pt')
    
    train_loss_total.append(train_loss_epoch)
    val_loss_total.append(val_loss_epoch)

print()
print('-'*50)
print(f'Finished Training ! Best Epoch is epoch {best_epoch+1}')

## 4. 모델 성능 평가

<img src='https://github.com/bogus215/LG-EDUCATION2/blob/main/image/image6.png?raw=1' width='500'></img>
<img src='https://github.com/bogus215/LG-EDUCATION2/blob/main/image/image7.png?raw=1' width='500'></img>

### 학습 상태 확인 (learning curve)

In [ ]:
plt.figure(figsize=(20,10))

# 학습 및 검증 로스 변동 관찰하기
plt.plot(train_loss_total,label='Train Loss')
plt.plot(val_loss_total, label='Validation Loss')
# 최적의 모델이 저장된 곳 표시
plt.axvline(x = best_epoch, color='red', label='Best Epoch')
plt.legend(fontsize=20)
plt.title("Learning Curve of trained model", fontsize=18)
plt.show()

In [ ]:
# 모델 선언
model = CNN_1d()
# 최적의 모델 불러오기
ckpt = torch.load('./best_model_HAR.pt')
# 모델 파라미터 불러오기
model.load_state_dict(ckpt)

In [ ]:
# model을 evaluation 모드로 변경
pred_list = []
true_list = []

model.eval()

with torch.no_grad():
    for inputs,labels in test_loader:
        # 모델의 출력값 계산
        y_logit = model(inputs)
        # 출력값을 최대로하는 인덱스(class) 저장
        y_pred = torch.argmax(y_logit, dim=1)

        # 예측값과 실제값 저장
        pred_list.extend(y_pred.detach().numpy())
        true_list.extend(labels.detach().numpy())

In [ ]:
# confusion matrix 계산
cm_train = confusion_matrix(y_true=true_list, y_pred=pred_list)

# confusion matrix 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(data=cm_train, annot=True, fmt='d', annot_kws={'size': 15}, cmap='Blues')
plt.xlabel('Predicted', size=20)
plt.ylabel('True', size=20)
plt.show()

# 평가지표 계산
test_acc = accuracy_score(true_list, pred_list)
test_rec = recall_score(true_list, pred_list, average='macro')
test_prec = precision_score(true_list, pred_list, average='macro')
test_f1 = f1_score(true_list, pred_list, average='macro')

print('Test Accuracy   : {:.3f}'.format(test_acc))
print('Test Sensitivity: {:.3f}'.format(test_rec))
print('Test Precision  : {:.3f}'.format(test_prec))
print('Test F1 Score   : {:.3f}'.format(test_f1))

### EOD